In [ ]:
# Freeze versions of dependencies for now
!pip install transformers==2.9.1
!pip3 install pytorch-lightning==0.8.4
!pip install -q aitextgen

import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

In [ ]:
ai = aitextgen(tf_gpt2="124M", to_gpu=True)

09/10/2020 13:28:56 — INFO — aitextgen — Downloading the 124M GPT-2 TensorFlow weights/config from Google's servers


09/10/2020 13:29:01 — INFO — aitextgen — Converting the 124M GPT-2 TensorFlow weights to PyTorch.



Save PyTorch model to aitextgen/pytorch_model.bin


09/10/2020 13:29:06 — INFO — aitextgen — Loading 124M GPT-2 model from /aitextgen.


Save configuration file to aitextgen/config.json


09/10/2020 13:29:11 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_name = 'reports.txt'

In [ ]:
copy_file_from_gdrive(file_name)

## Finetune GPT-2

Parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells
- **`learning_rate`**: Learning rate of the model training.
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM.

In [ ]:
ai.train(file_name,
         line_by_line=False,
         from_cache=False,
         num_steps=5000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=False,
         learning_rate=1e-4,
         batch_size=1, 
         )

09/10/2020 13:29:59 — INFO — aitextgen.TokenDataset — Encoding 12,907 sets of tokens from reports.txt.
GPU available: True, used: True
09/10/2020 13:30:00 — INFO — lightning — GPU available: True, used: True
TPU available: False, using: 0 TPU cores
09/10/2020 13:30:00 — INFO — lightning — TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
09/10/2020 13:30:00 — INFO — lightning — CUDA_VISIBLE_DEVICES: [0]


1,000 steps reached: saving model to /trained_model
1,000 steps reached: generating sample texts.
I ate several of the shrooms in a glass pipe before putting them on my tongue. I then put them in my mouth. I swallowed them, then put them on my tongue. I swallowed the rest. The rest of them were just about 2/3 as bad as the shrooms, so I think I ate about half. I really liked the taste of the shrooms, but I still ate a bit. I also did not like the taste of the shrooms. I did not like the taste of the shrooms, so I ate them again and gave them a go.

I took them with 2 other people, my boyfriend and I are planning on going on a big trip. We planned on going to a beach in Florida. We ate them with the rest of the party, and we had plenty of time to eat. So we ate the shrooms about four or five times. We split a half an eighth of the shrooms into half pieces, and ate about three of them at about five. We ate the rest of the bag, and all of the shrooms were swallowed. We ate the remaining t

09/10/2020 13:50:13 — INFO — aitextgen — Saving trained model pytorch_model.bin to /trained_model


.

It was like a complete revelation as to the nature of the universe.  I was in a completely different world.  I was in another place, a place where all possibilities seemed so far removed that I couldn't even imagine or trust.  I was on a completely different plane of perception and interaction.  I felt as though I had been in a completely different dimension.  I felt as though I had been inside a giant cookie-cut that was preventing some interaction on my level from existing in the first place.  I felt as though I had traveled much further than that and arrived at a new world.  I felt as though I had been lost in a labyrinth of forgotten places.  I felt as though I had been swallowed by a giant clock that was preventing me from coming back to my original reality.  

When I finally came back to my world (this was about 3 months later) I could see the colors and patterns in the trees and the flowers I had eaten before.  I also remember seeing the neon colors of blue and green in the c

# LICENSE

MIT License

Copyright (c) 2020 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.